# Twitter Sentiment Analysis v1

In [1]:
import twitter
import pandas as pd

### Source

https://towardsdatascience.com/creating-the-twitter-sentiment-analysis-program-in-python-with-naive-bayes-classification-672e5589a7ed

### Authenticating Twitter API

In [2]:
# Authenticating our twitter API credentials
twitter_api = twitter.Api(consumer_key='f2ujCRaUnQJy4PoiZvhRQL4n4',
                        consumer_secret='EjBSQirf7i83T7CX90D5Qxgs9pTdpIGIsVAhHVs5uvd0iAcw5V',
                        access_token_key='1272989631404015616-5XMQkx65rKfQU87UWAh40cMf4aCzSq',
                        access_token_secret='emfWcF8fyfqoyywfPCJnz4jXt6DFXfndro59UK9IMAMgy')

# Test authentication to make sure it was successful
print(twitter_api.VerifyCredentials())

{"created_at": "Tue Jun 16 20:29:26 +0000 2020", "default_profile": true, "default_profile_image": true, "id": 1272989631404015616, "id_str": "1272989631404015616", "name": "Nicola Osrin", "profile_background_color": "F5F8FA", "profile_image_url": "http://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png", "profile_image_url_https": "https://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png", "profile_link_color": "1DA1F2", "profile_sidebar_border_color": "C0DEED", "profile_sidebar_fill_color": "DDEEF6", "profile_text_color": "333333", "profile_use_background_image": true, "screen_name": "NicolaOsrin"}


### Building the Test Set

In [3]:
#We first build the test set, consisting of only 100 tweets for simplicity. 
#Note that we can only download 180 tweets every 15min.
def buildTestSet(search_keyword):
    try:
        tweets_fetched = twitter_api.GetSearch(search_keyword, count = 100)
        
        print("Fetched " + str(len(tweets_fetched)) + " tweets for the term " + search_keyword)
        
        return [{"text":status.text, "label":None} for status in tweets_fetched]
    except:
        print("Unfortunately, something went wrong..")
        return None

In [4]:
#Testing out fetching the test set. The function below prints out the first 5 tweets in our test set.
search_term = input("Enter a search keyword:")
testDataSet = buildTestSet(search_term)

print(testDataSet[0:4])

Enter a search keyword:stonks
Fetched 100 tweets for the term stonks
[{'text': 'STONKS https://t.co/lqgfsr29YD', 'label': None}, {'text': 'giveaway yey, honestly the best\nrules:\n\nfollow\nretweet\ncomment\nlike\nwin?\nstonks\n\nwinner sunday at some point in dm… https://t.co/d7dDHIsGpl', 'label': None}, {'text': 'Okay, I finally did it! I deposited the $10 my mom gave me in a Robinhood account and bought some stonks. I’m alrea… https://t.co/7HUftQOoEQ', 'label': None}, {'text': '@Alexandreeabreu Eu só perco na canastra pra mãe, stonks', 'label': None}]


In [6]:
#df = pd.DataFrame(list())
#df.to_csv('tweetDataFile.csv')

### Building the Training Set

We will be using a downloadable training set, consisting of 5,000 tweets. These tweets have already been labelled as positive/negative. We use this training set to calculate the posterior probabilities of each word appearing and its respective sentiment.

In [5]:
#As Twitter doesn't allow the storage of the tweets on personal drives, we have to create a function to download
#the relevant tweets that will be matched to the Tweet IDs and their labels, which we have.

def buildTrainingSet(corpusFile, tweetDataFile, size):
    import csv
    import time
    
    count = 0
    corpus = []
    
    with open(corpusFile,'r') as csvfile:
        lineReader = csv.reader(csvfile,delimiter=',', quotechar="\"")
        for row in lineReader:
            if count <= size: 
                corpus.append({"tweet_id":row[2], "label":row[1], "topic":row[0]})
                count += 1
            else: 
                break

    rate_limit = 180
    sleep_time = 900/180
    
    trainingDataSet = []
    
    for tweet in corpus:
        try:
            status = twitter_api.GetStatus(tweet["tweet_id"])
            print("Tweet fetched" + status.text)
            tweet["text"] = status.text
            trainingDataSet.append(tweet)
            time.sleep(sleep_time) 
        except: 
            continue
    # now we write them to the empty CSV file
    with open(tweetDataFile,'w') as csvfile:
        linewriter = csv.writer(csvfile,delimiter=',',quotechar="\"")
        for tweet in trainingDataSet:
            try:
                linewriter.writerow([tweet["tweet_id"], tweet["text"], tweet["label"], tweet["topic"]])
            except Exception as e:
                print(e)
    return trainingDataSet

In [6]:
#This function is used to download the actual tweets. It takes hours to run and we only need to run it once
#in order to get all 5,000 training tweets. The 'size' parameter below is the number of tweets that we want to
#download. If 5,000 => set size=5,000
'''
corpusFile = "corpus.csv"
tweetDataFile = "tweetDataFile.csv"

trainingData = buildTrainingSet(corpusFile, tweetDataFile, 1000)
'''

#When this code stops running, we will have a tweetDataFile.csv full of the tweets that we downloaded.

'\ncorpusFile = "corpus.csv"\ntweetDataFile = "tweetDataFile.csv"\n\ntrainingData = buildTrainingSet(corpusFile, tweetDataFile, 1000)\n'

In [8]:
#This line counts the number of tweets and their labels in the Corpus.csv file that we originally downloaded
corp = pd.read_csv("corpus.csv", header = 0 , names = ['topic','label', 'tweet_id'] )
corp['label'].value_counts()

neutral       2503
irrelevant    1786
negative       654
positive       569
Name: label, dtype: int64

In [9]:
#As a check, we look at the first 5 lines in our new tweetDataFile.csv
trainingData_copied = pd.read_csv("tweetDataFile.csv", header = None, names = ['tweet_id', 'text', 'label', 'topic'])
trainingData_copied.head()

,tweet_id,text,label,topic
0,126415614616154112,Now all @Apple has to do is get swype on the i...,positive,apple
1,126402758403305474,Hilarious @youtube video - guy does a duet wit...,positive,apple
2,126397179614068736,@RIM you made it too easy for me to switch to ...,positive,apple
3,126379685453119488,The 16 strangest things Siri has said so far. ...,positive,apple
4,126377656416612353,Great up close & personal event @Apple tonight...,positive,apple


In [10]:
#We check the number of tweets by each label in our training set
trainingData_copied['label'].value_counts()

neutral     316
negative    250
positive    114
Name: label, dtype: int64

In [11]:
trainingData_copied = trainingData_copied.to_dict('records')

### Pre-processing

Here we use the NLTK library to filter for keywords and remove irrelevant words in tweets. We also remove punctuation and things like images (emojis) as they cannot be classified using this model.

In [26]:
import re #a library that makes parsing strings and modifying them more efficient
from nltk.tokenize import word_tokenize
from string import punctuation 
from nltk.corpus import stopwords 
import nltk #Natural Processing Toolkit that takes care of any processing that we need to perform on text 
            #to change its form or extract certain components from it.
    
#nltk.download('popular') #We need this if certain nltk libraries are not installed. 

class PreProcessTweets:
    def __init__(self):
        self._stopwords = set(stopwords.words('english') + list(punctuation) + ['AT_USER','URL'])
        
    def processTweets(self, list_of_tweets):
        processedTweets=[]
        for tweet in list_of_tweets:
            processedTweets.append((self._processTweet(tweet["text"]),tweet["label"]))
        return processedTweets
    
    def _processTweet(self, tweet):
        tweet = tweet.lower() # convert text to lower-case
        tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
        tweet = re.sub('@[^\s]+', 'AT_USER', tweet) # remove usernames
        tweet = re.sub(r'#([^\s]+)', r'\1', tweet) # remove the # in #hashtag
        tweet = word_tokenize(tweet) # remove repeated characters (helloooooooo into hello)
        return [word for word in tweet if word not in self._stopwords]

In [27]:
#Here we call the function to pre-process both our training and our test set. 
tweetProcessor = PreProcessTweets()
preprocessedTrainingSet = tweetProcessor.processTweets(trainingData_copied)
preprocessedTestSet = tweetProcessor.processTweets(testDataSet)

### Building the Naive Bayes Classifier

We apply a classifier based on Bayes' Theorem, hence the name. It allows us to find the posterior probability of an event occuring (in this case that event being the sentiment- positive/neutral or negative) is reliant on another probabilistic background that we know. 

The posterior probability is calculated as follows:
$P(A|B) = \frac{P(B|A)\times P(A)}{P(B)}$

The final sentiment is assigned based on the highest probability of the tweet falling in each one.

### Build the vocabulary

In [30]:
#Here we attempt to build a vocabulary (a list of words) of all words present in the training set.

import nltk 

def buildVocabulary(preprocessedTrainingData):
    all_words = []
    
    for (words, sentiment) in preprocessedTrainingData:
        all_words.extend(words)

    wordlist = nltk.FreqDist(all_words)
    word_features = wordlist.keys()
    
    return word_features

#This function generates a list of all words (all_words) and then turns it into a frequency distribution (wordlist)
#The word_features is a list of distinct words, with the key being the frequency of each one.

### Matching tweets against our vocabulary

Here we go through all the words in the training set (i.e. our word_features list), comparing every word against the tweet at hand, associating a number with the word following:

label 1 (true): if word in vocabulary occurs in tweet

label 0 (false): if word in vocabulary does not occur in tweet

In [31]:
def extract_features(tweet):
    tweet_words = set(tweet)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in tweet_words)
    return features 

### Building our feature vector

In [32]:
word_features = buildVocabulary(preprocessedTrainingSet)
trainingFeatures = nltk.classify.apply_features(extract_features, preprocessedTrainingSet)

This feature vector shows if a particular tweet contains a certain word out of all the words present in the corpus in the training data + label (positive, negative or neutral) of the tweet.

We will feed this in the Naive Bayes Classifier, which will calculate the posterior probability given the prior probability of having a specific word in the tweet, and the likelihood of the specified outcome given the presence of this word based on the trainingFeatures data.

### Train the Naives Bayes Classifier

In [33]:
#This line trains our Bayes Classifier
NBayesClassifier = nltk.NaiveBayesClassifier.train(trainingFeatures)


## Test Classifier 

In [35]:
#We now run the classifier and test it on 100 tweets previously downloaded in the test set, on our specified keyword.

NBResultLabels = [NBayesClassifier.classify(extract_features(tweet[0])) for tweet in preprocessedTestSet]

# get the majority vote
if NBResultLabels.count('positive') > NBResultLabels.count('negative'):
    print("Overall Positive Sentiment")
    print("Positive Sentiment Percentage = " + str(100*NBResultLabels.count('positive')/len(NBResultLabels)) + "%")
else: 
    print("Overall Negative Sentiment")
    print("Negative Sentiment Percentage = " + str(100*NBResultLabels.count('negative')/len(NBResultLabels)) + "%")
    print("Positive Sentiment Percentage = " + str(100*NBResultLabels.count('positive')/len(NBResultLabels)) + "%")
    print("Number of negative comments = " + str(NBResultLabels.count('negative')))
    print("Number of positive comments = " + str(NBResultLabels.count('positive')))
    print("Number of neutral comments = " + str(NBResultLabels.count('neutral')))

Overall Negative Sentiment
Negative Sentiment Percentage = 4.0%
Positive Sentiment Percentage = 1.0%
Number of negative comments = 4
Number of positive comments = 1
Number of neutral comments = 95
